Find cross section of actors and movies?


In [1]:
// TODO: use papa parse to load tsv files from imdb
var util = require('util');
var fs = require('fs');
var papa = require('papaparse');
var sqlite3 = require('better-sqlite3');

var principals = '/Users/briancullinan/Downloads/title.principals.tsv';
var movies = './movies.sqlite.db';

var db = new sqlite3(movies, {});

function importSQL(file) {
    if(typeof file === 'string') {
        file = fs.createReadStream(file, { highWaterMark: 1024 * 1024 });
    }
    var prev = 0;
    var count = 0;
    return new Promise(resolve => papa.parse(file, {
        worker: true,
        delimiter: '\t',
        fastMode: true,
        step: function(results) {
            count++;
            if(count > prev + 1000) {
                prev = count;
                console.log(`${count} - ${results.data}`);
            }
            return db.prepare(`
INSERT OR IGNORE INTO principals
    (tconst, ordering, nconst, category, job, characters)
VALUES (?,?,?,?,?,?)`).run(results.data);
        },
        complete: resolve
    }));
}

function createTables() {
    db.prepare(`
CREATE TABLE IF NOT EXISTS principals (
    tconst TEXT NOT NULL,
    ordering INT NOT NULL,
    nconst TEXT NOT NULL,
    category TEXT,
    job TEXT,
    characters TEXT,
    UNIQUE(tconst,ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_titles
ON principals (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_names
ON principals (nconst);`).run()
    return Promise.resolve();
}

$$.async();
createTables()
    .then(() => importSQL(principals))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())



/*
params = [[1,2],[3,4],[5,6],[7,8]]

stmt = db.prepare("INSERT INTO test (num1, num2) VALUES (?1,?2)");
stmt.all([params], function(err, res) {
    ...
});
*/



1001 - tt0000557,3,nm1669708,actor,\N,["Captain Clearfield"]
2002 - tt0000760,4,nm1267460,actor,\N,\N
3003 - tt0000917,8,nm0601698,actor,\N,["A Boarder"]
4004 - tt0001077,9,nm0173994,actor,\N,["Museum Visitor"]
5005 - tt0001254,3,nm0038106,actress,\N,["Woman at Farewell"]
6006 - tt0001427,3,nm0629589,actor,\N,["Mr. Hendricks"]
7007 - tt0001600,4,nm0161366,actor,\N,\N
8008 - tt0001791,2,nm0440059,actor,\N,["A Modern Prince Charming"]
9009 - tt0001976,2,nm0165744,actress,\N,["Elizabeth Barton"]
10010 - tt0002144,3,nm0603011,production_designer,\N,\N
11011 - tt0002295,8,nm0480674,producer,producer,\N
12012 - tt0002459,2,nm0782962,actress,\N,["Mrs. Gregg"]
13013 - tt0002604,3,nm0700810,actor,\N,["H. Benjamin Baker - Florence's Father"]
14014 - tt0002767,6,nm0241416,writer,novel "Le Comte de Monte Cristo",\N
15015 - tt0002924,3,nm0635667,actress,\N,["Mabel"]
16016 - tt0003080,1,nm0822060,actor,\N,["Lord Harcourt - the Father"]
17017 - tt0003265,3,nm0827509,actor,\N,["Neighbor's Husband"]
18

152152 - tt0020498,10,nm0789049,cinematographer,\N,\N
153153 - tt0020610,6,nm0058882,writer,titles,\N
154154 - tt0020737,2,nm0111355,actress,\N,["Inez Sepulveda"]
155155 - tt0020853,8,nm0811033,writer,scenario,\N
156156 - tt0020977,1,nm0642988,actor,\N,["Bilge Smith"]
157157 - tt0021091,6,nm0089426,writer,play,\N
158158 - tt0021213,8,nm0712393,actress,\N,["New Zealand"]
159159 - tt0021353,4,nm0403154,actor,\N,["Gilbert Gaylord"]
160160 - tt0021474,3,nm0286721,actress,\N,["Mrs. Henderson"]
161161 - tt0021596,2,nm0281487,director,\N,\N
162162 - tt0021732,3,nm0124236,actor,\N,["Tom Ingleside"]
163163 - tt0021855,4,nm0142120,actress,\N,["Tante Aurélie"]
164164 - tt0021973,10,nm0183918,actress,\N,["Margie"]
165165 - tt0022094,8,nm0788356,writer,adaptation,\N
166166 - tt0022211,2,nm0287988,actor,\N,["Douglas Thayer"]
167167 - tt0022334,4,nm0954647,actress,\N,["Agnes Andersson"]
168168 - tt0022449,4,nm0040341,actor,\N,["Mr. Marston"]
169169 - tt0022574,4,nm0449698,actor,\N,["Allen Greenough"]

304304 - tt0038116,3,nm0371376,actor,\N,["Wayne Frake"]
305305 - tt0038232,4,nm0898085,actor,\N,["Don Agustín"]
306306 - tt0038346,1,nm0484808,actress,\N,["Janet Bradley"]
307307 - tt0038455,2,nm0000817,actor,\N,["Dr. Scott Elliott"]
308308 - tt0038566,10,nm0438443,production_designer,\N,\N
309309 - tt0038682,1,nm0064604,actor,\N,["Michael Shayne"]
310310 - tt0038795,5,nm0332539,director,\N,\N
311311 - tt0038913,3,nm0577523,actor,\N,["Raffaele"]
312312 - tt0039021,3,nm0167395,actor,\N,["Andy Warren"]
313313 - tt0039135,5,nm0168413,director,\N,\N
314314 - tt0039248,10,nm0554921,cinematographer,\N,\N
315315 - tt0039357,7,nm0448200,writer,novel and play,\N
316316 - tt0039472,3,nm0534972,actor,\N,["Corporal Phil Vaughn"]
317317 - tt0039583,10,nm0005803,cinematographer,director of photography,\N
318318 - tt0039696,10,nm0953790,writer,\N,\N
319319 - tt0039822,2,nm0836677,actress,\N,["La nièce"]
320320 - tt0039931,4,nm0000472,actor,\N,["Guyasuta - Chief of the Senecas"]
321321 - tt0040057,3,n

457457 - tt0055462,8,nm0005652,cinematographer,\N,\N
458458 - tt0055572,4,nm0040933,actor,\N,["Faux aveugle #2"]
459459 - tt0055684,2,nm1813453,self,\N,["Themselves"]
460460 - tt0055794,1,nm0774622,actress,\N,["Pyret Sträng"]
461461 - tt0055906,7,nm0595063,writer,\N,\N
462462 - tt0056017,2,nm0441251,actress,\N,["Colette"]
463463 - tt0056127,7,nm0322874,composer,\N,\N
464464 - tt0056237,8,nm0686469,composer,\N,\N
465465 - tt0056346,9,nm0622434,cinematographer,director of photography,\N
466466 - tt0056455,6,nm0309265,writer,\N,\N
467467 - tt0056568,9,nm0005807,cinematographer,\N,\N
468468 - tt0056681,4,nm0041681,actor,\N,["Colonel & President"]
469469 - tt0056791,6,nm0000314,actor,\N,["Linc Murdock"]
470470 - tt0056902,10,nm0006274,composer,\N,\N
471471 - tt0057007,8,nm0069410,writer,story,\N
472472 - tt0057119,8,nm0928052,cinematographer,\N,\N
473473 - tt0057232,8,nm0958309,composer,\N,\N
474474 - tt0057340,1,nm1954808,actress,\N,\N
475475 - tt0057450,2,nm0286227,producer,producer,\N
47

611611 - tt0072823,9,nm0005996,composer,\N,\N
612612 - tt0072935,7,nm2049863,composer,\N,\N
613613 - tt0073052,5,nm0206646,director,\N,\N
614614 - tt0073172,5,nm0126440,director,\N,\N
615615 - tt0073288,8,nm0767902,writer,deviser,\N
616616 - tt0073408,2,nm0340039,actor,\N,["Gerry"]
617617 - tt0073530,1,nm0001958,actor,\N,["Alan Curtis"]
618618 - tt0073647,9,nm0134821,actor,\N,\N
619619 - tt0073760,4,nm0003110,actor,\N,["A.J. Arno"]
620620 - tt0073875,5,nm0496312,director,\N,\N
621621 - tt0073991,2,nm0939712,self,\N,["Himself - Announcer"]
622622 - tt0074102,8,nm0586523,composer,\N,\N
623623 - tt0074220,10,nm0687360,production_designer,\N,\N
624624 - tt0074330,4,nm0675490,actor,\N,["Jesse Jerome"]
625625 - tt0074445,1,nm0417923,actress,\N,["Aniela Dulska"]
626626 - tt0074560,2,nm0266028,actor,\N,["Récitant"]
627627 - tt0074674,8,nm0548903,producer,producer,\N
628628 - tt0074786,3,nm0062222,actor,\N,["Albert"]
629629 - tt0074903,9,nm0598723,producer,producer,\N
630630 - tt0075016,5,nm082

...

33639606 - tt9760742,1,nm0576775,actress,\N,["Laura Selmhorst"]
33640607 - tt9761328,7,nm6184490,writer,\N,\N
33641608 - tt9761568,5,nm4758523,writer,writer,\N
33642609 - tt9761798,10,nm5387155,producer,producer,\N
33643610 - tt9762030,7,nm4779832,writer,\N,\N
33644611 - tt9762310,5,nm1729216,director,\N,\N
33645612 - tt9762584,6,nm7480673,writer,\N,\N
33646613 - tt9762802,2,nm4861949,actress,\N,["Pallavi Shah"]
33647614 - tt9763116,7,nm0438471,producer,producer,\N
33648615 - tt9763358,2,nm5013143,actor,\N,["Samrat Shergill"]
33649616 - tt9763682,3,nm10472462,director,\N,\N
33650617 - tt9764172,8,nm0656951,cinematographer,\N,\N
33651618 - tt9764624,1,nm3762685,actor,\N,\N
33652619 - tt9765170,2,nm1769807,self,\N,["Himself"]
33653620 - tt9765714,2,nm5673777,actor,\N,["Hydorous"]
33654621 - tt9766160,9,nm10469087,actor,\N,\N
33655622 - tt9766888,5,nm5391776,director,\N,\N
33656623 - tt9767140,6,nm9191439,self,\N,["Herself - Co-Host"]
33657624 - tt9767386,4,nm2189240,self,\N,["Himself - C

33789756 - tt9816066,9,nm0847351,actress,\N,["Reiko Shiratori"]
33790757 - tt9816572,1,nm2455676,self,\N,["Himself"]
33791758 - tt9817018,5,nm3492802,director,\N,\N
33792759 - tt9817566,7,nm2066600,actor,\N,["Patrick Bissonnette"]
33793760 - tt9818032,1,nm0454730,self,\N,["Herself - Co-Host"]
33794761 - tt9818386,9,nm0694900,actress,\N,["Sudha's mother (1998-1999)"]
33795762 - tt9818680,1,nm9473360,actress,\N,["Saroj Tilak Chopra"]
33796763 - tt9818984,3,nm1431194,director,\N,\N
33797764 - tt9819288,7,nm1026557,composer,\N,\N
33798765 - tt9819536,9,nm1199935,writer,\N,\N
33799766 - tt9819788,3,nm7207144,director,\N,\N
33800767 - tt9820030,2,nm1817397,actress,\N,["MahaMai"]
33801768 - tt9820270,1,nm1110573,actress,\N,["Padmaja a.k.a Pappa"]
33802769 - tt9820646,1,nm5803911,self,\N,["Himself - Host"]
33803770 - tt9821168,9,nm6014536,actress,\N,\N
33804771 - tt9821530,5,nm10494631,writer,\N,\N
33805772 - tt9821784,6,nm0747025,director,\N,\N
33806773 - tt9822002,9,nm1428113,cinematographer

33936903 - tt9872350,6,nm5965405,self,\N,["Himself - Guest"]
33937904 - tt9872824,5,nm9818935,director,\N,\N
33938905 - tt9873110,1,nm4449367,actress,\N,["Pankti Sharma"]
33939906 - tt9873330,5,nm10507252,writer,\N,\N
33940907 - tt9873636,2,nm0536821,actress,\N,["Ranjana Devi"]
33941908 - tt9873976,3,nm3258181,self,\N,["Himself - Deputy Political Editor"]
33942909 - tt9874324,1,nm7129302,actor,\N,["Siddharth Singh Khurana"]
33943910 - tt9874724,8,nm5744243,production_designer,\N,\N
33944911 - tt9875142,10,nm0720105,self,\N,["Himself"]
33945912 - tt9875632,4,nm2951723,archive_footage,\N,["Himself - Manhattan Institute"]
33946913 - tt9876086,6,nm9936326,archive_footage,\N,["Herself - New York Congresswoman"]
33947914 - tt9876558,3,nm10520599,self,\N,["Herself"]
33948915 - tt9877246,2,nm2017096,self,\N,["Himself"]
33949916 - tt9878056,5,nm8446837,self,\N,["Himself"]
33950917 - tt9878944,2,nm1290102,actress,\N,["Mariko Yashida"]
33951918 - tt9879276,1,nm1258869,actress,\N,["Hasmin"]
339529

{ data: [], errors: [], meta: {} }

Database {
  inTransaction: false,
  open: false,
  memory: false,
  readonly: false,
  name: './movies.sqlite.db' }